# Experiments for evaluating the explanations of the Group Recommender System

1. Ask the user the same questionnaire as the individual experiment. But do not recommend. 
2. Instead add the user to the user matrix for group recommender system.
3. Now start the group recommender setup as usual; fill in the NaN with knn collaboartive filtering, make 4 clusters, and make groups. 
4. Display the explanations for the group in which the user we created initially is in. 

In [1]:
import csv
import pandas as pd
import numpy as np
import tkinter as tk
from lenskit import batch, topn
from nltk.corpus import stopwords
from tkinter import messagebox
from lenskit import crossfold as xf
from nltk.tokenize import word_tokenize
from lenskit.metrics import topn as tnmetrics
from lenskit.algorithms import Recommender
from lenskit.algorithms.user_knn import UserUser
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
# Imports 
import pandas as pd
import numpy as np
import random
import itertools
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
from lenskit import topn
from lenskit import crossfold as xf
from lenskit import batch, topn, util
from lenskit.algorithms import Recommender, user_knn as uknn
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn_extra.cluster import KMedoids
from sklearn.model_selection import train_test_split
warnings.filterwarnings(action='ignore', category=RuntimeWarning, module='lenskit.metrics.topn')
warnings.filterwarnings(action='ignore', category=FutureWarning, module='lenskit.metrics.topn')
from surprise.model_selection import cross_validate
from lenskit.algorithms import Recommender, bias, basic, item_knn
from lenskit.algorithms.user_knn import UserUser 

In [2]:
preprocessed_dataset_folder = "../Data/PreprocessedDataset"
ratings_df = pd.read_csv(preprocessed_dataset_folder+"/ratings.csv")
movies_df = pd.read_csv(preprocessed_dataset_folder+"/movies.csv")
user_plots_ratings_df = pd.read_csv(preprocessed_dataset_folder+"/user_plots.csv") #first run notebook algorithm_experiments to get this data

In [5]:
def least_misery_strategy(group, n_movies):

    if 'cluster' in group.columns:
        group = group.drop(columns=['cluster'])

    min_ratings = group.min(axis=0)
    sorted_movies = min_ratings.sort_values(ascending=False)
    return sorted_movies.head(n_movies)


In [6]:
def most_pleasure_strategy(group, n_movies):

    if 'cluster' in group.columns:
        group = group.drop(columns=['cluster'])

    max_ratings = group.max(axis=0)
    sorted_movies = max_ratings.sort_values(ascending=False)
    return sorted_movies.head(n_movies)

In [7]:
def majority_strategy(group, n_movies, threshold=4): 
    print(group)
    likes = group[group >= threshold]
    like_counts = likes.count()
    majority_threshold = len(group) / 2
    majority_likes = like_counts[like_counts > majority_threshold]
    return majority_likes.nlargest(n_movies)
        

In [8]:
def approval_voting_strategy(group, n_movies):         
    if 'cluster' in group.columns:
        group = group.drop(columns=['cluster'])

    approval_threshold = 4 
    approved_items_count = (group > approval_threshold).sum(axis=0)
    sorted_movies = approved_items_count.sort_values(ascending=False)
    
    return (sorted_movies.head(n_movies))
      

In [9]:
def additive_utilitarian_strategy(group, n_movies):

    if 'cluster' in group.columns:
        group = group.drop(columns=['cluster'])

    total_ratings = group.sum(axis=0)
    sorted_movies = total_ratings.sort_values(ascending=False)
    return (sorted_movies.head(n_movies))


In [10]:
def minority_groups(clusters, num_of_groups, group_size=4):
    all_groups = {}
    group_count = 0

    viable_clusters = [group for _, group in clusters if len(group) >= 1]

    while group_count < num_of_groups:
        for cluster_combination in itertools.permutations(viable_clusters, 2):
            cluster_a, cluster_b = cluster_combination
            if len(cluster_a) >= group_size - 1 and len(cluster_b) >= 1:
                sampled_users_a = cluster_a.sample(n=group_size - 1)
                sampled_user_b = cluster_b.sample(n=1)
                group_of_sampled_users = pd.concat([sampled_users_a, sampled_user_b])
                all_groups[f'group_{group_count+1}'] = group_of_sampled_users
                group_count += 1

                if group_count >= num_of_groups:
                    break

    return all_groups

In [11]:
def coalitional_groups(clusters, user_id):
    user_cluster_id = None
    for cluster_id, group in clusters:
        if user_id in group.index:
            user_cluster_id = cluster_id
            break


    user_cluster_df = [group for cluster_id, group in clusters if cluster_id == user_cluster_id][0]
    sampled_user_from_same_cluster = user_cluster_df.drop(user_id).sample(n=1)
    other_clusters = [group for cluster_id, group in clusters if cluster_id != user_cluster_id]
    chosen_cluster = random.choice(other_clusters)  # Randomly choose another cluster
    sampled_users_from_other_cluster = chosen_cluster.sample(n=2)
    group = pd.concat([user_cluster_df.loc[[user_id]], sampled_user_from_same_cluster, sampled_users_from_other_cluster])
    return group

In [12]:

def minority_groups(clusters, user_id):
    user_cluster_id = None
    for cluster_id, group in clusters:
        if user_id in group.index:
            user_cluster_id = cluster_id
            break

    other_clusters = [group for cluster_id, group in clusters if cluster_id != user_cluster_id]
    chosen_cluster = random.choice(other_clusters)
    sampled_users = chosen_cluster.sample(n=3)
    user_cluster_df = [group for cluster_id, group in clusters if cluster_id == user_cluster_id][0]
    group = pd.concat([user_cluster_df.loc[[user_id]], sampled_users])

    return group

In [13]:
def uniform_group(clusters, user_id):
    user_cluster = None
    for cluster_id, group in clusters:
        if user_id in group.index:
            user_cluster = group
            break
    if user_cluster is None:
        raise ValueError(f"User ID {user_id} not found in any cluster.")
    sampled_users = user_cluster.drop(user_id).sample(n=3)
    uniform_group = pd.concat([user_cluster.loc[[user_id]], sampled_users])

    return uniform_group

In [14]:
def divergent_groups(clusters, num_of_groups, user_id):
    all_groups = {}

    user_cluster_id = None
    for cluster_id, group in clusters:
        if user_id in group.index:  
            user_cluster_id = cluster_id
            break
    for i in range(num_of_groups):
        sampled_users = []
        for cluster_id, group in clusters:
            if not group.empty:
                if cluster_id == user_cluster_id:
                    sampled_users.append(group.loc[[user_id]])
                else:
                    sampled_user = group.sample(n=1)
                    sampled_users.append(sampled_user)

        group_of_sampled_users = pd.concat(sampled_users)
        all_groups[f'group_{i+1}'] = group_of_sampled_users

    return all_groups

In [15]:
def get_movie_title(movie_id):
    movie_row = movies_df[movies_df['item'] == movie_id]
    return movie_row.iloc[0]['title']


def generate_explanations(group, aggregation_strat, n_of_recommendations):
    explanations = list()

    if(aggregation_strat == "ADD"):
        group_results = additive_utilitarian_strategy(group ,n_of_recommendations)
        recommended_movies = ', '.join(map(lambda id_: f"'{get_movie_title(id_)}'", group_results.index))
        if(len(group_results)>1):
            explanations.append(f"No explanation: We advise your group to consider movies {recommended_movies}.")
            explanations.append(f"Basic explanation: Movies {recommended_movies} have been recommended to your group because they hold the maximum cumulative rating.")
            explanations.append(f"Detailed explanation: Movies {recommended_movies} have been recommended to your group because they hold the maximum cumulative rating. This means after adding all group members ratings’ for all movies, movies {recommended_movies} have the highest sum.")
        else:
            explanations.append(f"No explanation: We advise your group to consider movie {recommended_movies}.")
            explanations.append(f"Basic explanation: Movie {recommended_movies} has been recommended to your group because it holds the maximum cumulative rating.")
            explanations.append(f"Detailed explanation: Movie {recommended_movies} has been recommended to your group because it holds the maximum cumulative rating. This means after adding all group members ratings’ for all movies, movie {recommended_movies} had the highest sum.")
    if(aggregation_strat == "APP"):
        group_results = approval_voting_strategy(group, n_of_recommendations)
        recommended_movies = ', '.join(map(lambda id_: f"'{get_movie_title(id_)}'", group_results.index))
        if(len(group_results)>1):
            explanations.append(f"No explanation: We advise your group to consider movies {recommended_movies}.")
            explanations.append(f"Basic explanation: Movies {recommended_movies} have been recommended to your group because they have the highest amount of ratings above threshold θ.")
            explanations.append(f"Detailed explanation: Movies {recommended_movies} have been recommended to your group because they have the highest amount of ratings above threshold θ . This means after gathering all group members ratings for each movie, movies {recommended_movies} have the most ratings above θ.")
        else:
            explanations.append(f"No explanation: We advise your group to consider movie {recommended_movies}.")
            explanations.append(f"Basic explanation: Movie {recommended_movies} has been recommended to your group because it has the highest amount of ratings above threshold θ.")
            explanations.append(f"Detailed explanation: Movie {recommended_movies} has been recommended to your group because it has the highest amount of ratings above threshold θ . This means after gathering all group members ratings for each movie, movie {recommended_movies} had the most ratings above θ.")
    if(aggregation_strat == "LMS"):
        group_results = least_misery_strategy(group, n_of_recommendations)
        recommended_movies = ', '.join(map(lambda id_: f"'{get_movie_title(id_)}'", group_results.index))
        if(len(group_results)>1):
            explanations.append(f"No explanation: We advise your group to consider movies {recommended_movies}.")
            explanations.append(f"Basic explanation: Movies {recommended_movies} have been recommended to your group as they don't pose significant issues for any member.")
            explanations.append(f"Detailed explanation: Movies {recommended_movies} have been recommended to your group as they don't pose significant issues for any member. This means that for all movies rated, the highest ratings amongst all lowest ratings per movie was the lowest rating of movies {recommended_movies}.")
        else:
            explanations.append(f"No explanation: We advise your group to consider movie {recommended_movies}.")
            explanations.append(f"Basic explanation: Movie {recommended_movies} has been recommended to your group as it doesn't pose significant issues for any member.")
            explanations.append(f"Detailed explanation: Movie {recommended_movies} has been recommended to group your group as it doesn't pose significant issues for any member. This means that for all movies rated, the highest rating amongst all lowest ratings per movie was the lowest rating of movie {recommended_movies}.")
    if(aggregation_strat == "MPL"):
        group_results = most_pleasure_strategy(group, n_of_recommendations)
        recommended_movies = ', '.join(map(lambda id_: f"'{get_movie_title(id_)}'", group_results.index))
        if(len(group_results)>1):
            explanations.append(f"No explanation: We advise your group to consider movies {recommended_movies}.")
            explanations.append(f"Basic explanation: Movies {recommended_movies} are recommended to your group as they have received the highest ratings from all members.")
            explanations.append(f"Detailed explanation: Movies {recommended_movies} are recommended to your group as they have received the highest ratings from all members. This means that for the highest ratings from all group members for each movie, movies {recommended_movies} had the highest rating.")
        else:
            explanations.append(f"No explanation: We advise group {int(group_id.split('_')[-1])} to consider movie {recommended_movies}.")
            explanations.append(f"Basic explanation: Movie {recommended_movies} is recommended to your group as it has received the highest ratings from all members.")
            explanations.append(f"Detailed explanation: Movie {recommended_movies} is recommended to your group as it has received the highest ratings from all members. This means that for the highest ratings from all group members for each movie, movie {recommended_movies} had the highest rating.")
    if(aggregation_strat == "MAJ"):
        group_results = majority_strategy(group, n_of_recommendations)
        recommended_movies = ', '.join(map(lambda id_: f"'{get_movie_title(id_)}'", group_results.index))
        if(len(group_results)>1):
            explanations.append(f"No explanation: We advise your group to consider movies {recommended_movies}.")
            explanations.append(f"Basic explanation: Movies {recommended_movies} are suggested to your group as they are favored by the majority of members.")
            explanations.append(f"Detailed explanation: Movies {recommended_movies} are suggested to your group as  they are favored by the majority of members. This means the majority of the members in the group gave them a high rating.")
        else:
            explanations.append(f"No explanation: We advise your group to consider movie {recommended_movies}.")
            explanations.append(f"Basic explanation: Movie {recommended_movies} is suggested to your group as it is favored by the majority of members.")
            explanations.append(f"Detailed explanation: Movie {recommended_movies} is suggested to your group as it is favored by the majority of members. This means the majority of the members in the group gave it a high rating.")

    return explanations

In [16]:
def getExplanations(recommended_movies, cosine_sim_df,user_plots_ratings_df, user_id,k=3):
    explanations = []
    for index, row in recommended_movies.iterrows():
        recommended_item_id = row['item']
        item_title = row['title']
        string = f"Recommended item \" {item_title}\" because you previously watched items with similar plot:"
        explanations.append(string)
        print(string)
        cosine_sim_values = cosine_sim_df.loc[recommended_item_id].sort_values(ascending=False)
        
        similar_items = cosine_sim_values.index[0:k]
        
        similar_items_info = []
        for similar_item_id in similar_items:
            similar_item_title = movies_df[movies_df['item'] == similar_item_id]['title'].values[0]
            similar_item_rating = user_plots_ratings_df[(user_plots_ratings_df['item'] == similar_item_id) & (user_plots_ratings_df['user_id'] == user_id)]['rating'].values[0]
            similar_items_info.append((similar_item_id, similar_item_title, similar_item_rating))
        
        for i, (similar_item_id, similar_item_title, similar_item_rating) in enumerate(similar_items_info):
            string2 = f" {i+1})  \"{similar_item_title}\" (ID: {similar_item_id}) and gave rating: {similar_item_rating}"
            explanations.append(string2)
            print(string2)
    return explanations

In [18]:
def load_dataset():
    preprocessed_dataset_folder = "../Data/PreprocessedDataset"
    ratings_df = pd.read_csv(preprocessed_dataset_folder+"/ratings.csv")
    movies_df = pd.read_csv(preprocessed_dataset_folder+"/movies.csv")
    user_plots_ratings_df = pd.read_csv(preprocessed_dataset_folder+"/user_plots.csv") #first run notebook algorithm_experiments to get this data
    return ratings_df, movies_df, user_plots_ratings_df

In [19]:
def k_means(n_of_clusters,user_matrix_filled,n_runs=100):

    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(user_matrix_filled)

    best_centers = None
    best_labels = None
    min_inertia = float('inf')

    for i in range(n_runs):
        kmeans = KMeans(n_clusters=n_of_clusters, random_state=i).fit(scaled_data)
        if kmeans.inertia_ < min_inertia:
            min_inertia = kmeans.inertia_
            best_centers = kmeans.cluster_centers_
            best_labels = kmeans.labels_

    user_matrix_filled['cluster'] = best_labels
    grouped = user_matrix_filled.groupby('cluster')

    return best_centers, grouped

In [20]:
def collaborative_filtering(data):
    algo = UserUser(15,min_nbrs=3) 
    algo = Recommender.adapt(algo)
    algo.fit(data)

    user_ids = data['user'].unique()  
    item_ids = data['item'].unique() 

    all_preds = []

    for user in user_ids:
        preds = algo.predict_for_user(user, item_ids)
        for item, pred in zip(item_ids, preds):
            if not np.isnan(pred): 
                all_preds.append([user, item, pred])

    pred_df = pd.DataFrame(all_preds, columns=['user', 'item', 'rating'])
    filled_user_matrix = pred_df.pivot(index='user', columns='item', values='rating')
    filled_user_matrix.fillna(0, inplace=True)
    return filled_user_matrix



In [21]:
def preprocess_plot_text(plot_text):
    tokens = word_tokenize(plot_text.replace("|"," ")) # to split genres
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word.lower() for word in tokens if word.isalnum() and word.lower() not in stop_words]
    return " ".join(filtered_tokens)

movies_df['plot + title + genres'] = (movies_df['plot']+" "+movies_df['title']+" "+movies_df['genres']).apply(preprocess_plot_text)
def get_user_rated_movies_plots(new_user_df, movies):
    rated_item_ids = list(movies)
    rated_movies_df = movies_df[movies_df['item'].isin(rated_item_ids)]
    rated_movies_df = pd.merge(rated_movies_df[['item', 'plot + title + genres']], new_user_df, on='item')
    rated_movies_df['user_id'] = 0
    return rated_movies_df

In [23]:
class MovieRatingGUI:
    def __init__(self, root, movies):
        self.root = root
        self.movies = movies
        self.ratings = {}
        self.create_movie_rating_form(submit_button_size=(20, 5), submit_button_position=(600, 300))

    def create_movie_rating_form(self, submit_button_size, submit_button_position):
        enlarged_width = 800  
        enlarged_height = 600

        self.root.geometry(f"{enlarged_width}x{enlarged_height}")
        title_label = tk.Label(self.root, text="Pick at least 15 movies which you watched:", font=("Arial", 20))
        title_label.pack()
        scrollbar = tk.Scrollbar(self.root, orient=tk.VERTICAL)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        self.movie_listbox = tk.Listbox(self.root, yscrollcommand=scrollbar.set, selectmode=tk.MULTIPLE, font=("Arial", 15), height=20, width=50)
        for movie in self.movies:
            self.movie_listbox.insert(tk.END, movie)
        self.movie_listbox.pack(side=tk.LEFT, padx=20, pady=20)
        scrollbar.config(command=self.movie_listbox.yview)
        submit_button = tk.Button(self.root, text="Submit", command=self.submit_ratings, width=submit_button_size[0], height=submit_button_size[1], bg="red")
        submit_button.place(x=submit_button_position[0], y=submit_button_position[1])

    def submit_ratings(self):
        selected_movies_indices = self.movie_listbox.curselection()
        if len(selected_movies_indices) < 15:
            messagebox.showerror("Error", "Please select at least 15 movies.")
        else:
            selected_movies = [self.movies[index] for index in selected_movies_indices]
            if len(selected_movies) > 15:
                selected_movies = selected_movies[:15]  
            self.show_selected_movies(selected_movies)

    def show_selected_movies(self, movies):
        new_window = tk.Tk()
        new_window.title("Rate Selected Movies")
        movie_ratings = {}

        for movie in movies:
            label = tk.Label(new_window, text=movie, font=("Arial", 10))
            label.grid(row=movies.index(movie), column=0, sticky="w", padx=10, pady=1)
            rating_scale = tk.Scale(new_window, from_=0, to=5, orient=tk.HORIZONTAL, resolution=0.1)
            rating_scale.grid(row=movies.index(movie), column=1, padx=10, pady=1)
            movie_ratings[movie] = rating_scale

        submit_button = tk.Button(new_window, text="Submit Ratings", command=lambda: self.submit_ratings_with_ratings(new_window, movie_ratings))
        submit_button.grid(row=len(movies), columnspan=2, pady=10)
 
    def submit_ratings_with_ratings(self, window, movie_ratings):
        data, movies_df, user_plots_ratings_df = load_dataset()
        new_user = [{'title': movie, 'rating': float(rating.get())} for movie, rating in movie_ratings.items()]
        new_user_df = pd.DataFrame(new_user)
        print(new_user_df)
        
        selected_titles = new_user_df['title'].tolist()
        filtered_movies_df = movies_df[movies_df['title'].isin(selected_titles)]
        new_user_df = pd.merge(new_user_df, filtered_movies_df[['item', 'title']], on='title')
        user_plot = get_user_rated_movies_plots(new_user_df, filtered_movies_df['item'])
        user_plot_all = pd.concat([user_plots_ratings_df, user_plot], axis=0)
        new_user_id = data['user'].max() + 1
        rows_to_add = []
        for index, entry in user_plot_all.iterrows():
            movie_id = movies_df[movies_df['title'] == entry["title"]]['item'].values

            if len(movie_id) > 0: 
                rows_to_add.append({
                    'user': new_user_id,
                    'item': movie_id[0],
                    'rating': entry['rating'],
                    'timestamp': 0 
                })

        new_user_data = pd.DataFrame(rows_to_add)
        data = pd.concat([data, new_user_data])       
        user_matrix_filled = collaborative_filtering(data)    

        best_centers, grouped = k_means(4, user_matrix_filled)
        #groups = minority_groups(grouped, new_user_id, 1) 
        #groups = coalitional_groups(grouped, new_user_id, 1) 
        #groups = divergent_groups(grouped, new_user_id, 1) 
        group = uniform_group(grouped,  new_user_id) 
        explanations = generate_explanations(group, "LMS", 1)

        explanation_window = tk.Tk()
        explanation_window.title("Rate explanations")
        window_width = 1000
        window_height = 800
        screen_width = explanation_window.winfo_screenwidth()
        screen_height = explanation_window.winfo_screenheight()

        x_coordinate = (screen_width - window_width) // 2
        y_coordinate = (screen_height - window_height) // 2
    
        explanation_window.geometry(f"{window_width}x{window_height}+{x_coordinate}+{y_coordinate}")
        explanations_listbox = tk.Listbox(explanation_window, font=("Arial", 12), width=80, height=20)
        explanations_listbox.grid(row=0, column=0, padx=20, pady=20)

        for explanation in explanations:
            explanations_listbox.insert(tk.END, explanation)

        questions = [
            "Add something here.",
            "Add something here.",
            "Add something here."
        ]

        rating_scales = []
        current_row = 1  
        for question in questions:
            label = tk.Label(explanation_window, text=question, wraplength=800)
            label.grid(row=current_row, column=0, padx=20, pady=5, sticky=tk.W)
            current_row += 1

            scale = tk.Scale(explanation_window, from_=0, to=5, orient=tk.HORIZONTAL, resolution=0.1, length=200)
            scale.grid(row=current_row, column=0, padx=20, pady=5)
            current_row += 1

            rating_scales.append(scale)

        label_question = tk.Label(explanation_window, text="Add something here.", wraplength=800)
        label_question.grid(row=current_row, column=0, padx=20, pady=5, sticky=tk.W)
        current_row += 1
        class ToggleButton(tk.Button):
            def __init__(self, parent, **kwargs):
                self._values = ["1", "3"]
                self._current_index = 0
                super().__init__(parent, text=self._values[self._current_index], command=self.toggle, **kwargs)

            def toggle(self):
                self._current_index = 1 - self._current_index 
                self.config(text=self._values[self._current_index])

            def get_value(self):
                return self._values[self._current_index]

        toggle_button = ToggleButton(explanation_window, width=5)
        toggle_button.grid(row=current_row, column=0, padx=20, pady=5, sticky=tk.W)
        current_row += 1

        def submit_and_close():
            ratings = [scale.get() for scale in rating_scales]
            yes_no_answer = yes_no_answer = toggle_button.get_value()
            print(yes_no_answer)
            with open("../Data/PreprocessedDataset/feedback.csv", 'a', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(ratings + [yes_no_answer])

        submit_button_2 = tk.Button(explanation_window, text="Submit Ratings", command=submit_and_close)
        submit_button_2.grid(row=current_row, column=0, pady=10)
            
movies_list =  movies_df['title'].values.tolist()
root = tk.Tk()
root.title("Movie Rating App")
app = MovieRatingGUI(root, movies_list)
root.mainloop()

                          title  rating
0   father of the bride part ii     0.0
1                       sabrina     0.0
2                     goldeneye     0.0
3                         nixon     0.0
4         sense and sensibility     0.0
5                    get shorty     0.0
6                       copycat     0.0
7             leaving las vegas     0.0
8               dangerous minds     0.0
9              dead man walking     0.0
10                  richard iii     5.0
11                mortal kombat     0.0
12                   to die for     0.0
13                   pocahontas     0.0
14                     bio-dome     0.0
